In [1]:
import numpy as np
import sys
sys.path.append( '../vqe-in-dft' )

import vqe_in_dft 

import scipy as sp
from pyscf import gto, dft, lib, mp, cc, scf, tools, ci, fci

Basis options:
https://pyscf.org/_modules/pyscf/gto/basis.html

In [10]:
### methane
# https://github.com/nutjunkie/IQmol/blob/master/share/fragments/Molecules/Alkanes/Methane.xyz
geometry = [
['C', (0.0000, 0.0000, 0.0000)],
['H', (0.5288, 0.1610, 0.9359)],
['H', (0.2051, 0.8240, -0.6786)],
['H', (0.3345, -0.9314, -0.4496)],
['H', (-1.0685, -0.0537, 0.1921)]
]
    
### water
# geometry = [
# ['H', (0.7493682,0.0000000,0.4424329)],
# ['O', (0.0000000,0.0000000,-0.1653507)],
    
# ['H', (-0.7493682,0.0000000,0.4424329)]
# ]


### ethanol
# geometry = [
# ['H', (1.2473876659, -0.8998737590, 0.6150681570)],
# ['O', (1.2322305822, -0.2731895077, -0.1276123902)],

# ['C', (0.0849758188, 0.5590385475, 0.0510545434)],
# ['H', (0.1506137362, 1.1200249874, 0.9943015309)],
# ['H', (0.1316093068, 1.2841805400, -0.7645223601)],

# ['C', (-1.2129704155, -0.2295285634, -0.0097156258)],
# ['H', (-2.0801425360, 0.4329727646,0.0722817289)],
# ['H', (-1.2655910941, -0.9539857247, 0.8097953440)],
# ['H', (-1.2737541560, -0.7748626513, -0.9540587845)],
# ]

In [11]:


N_active_atoms = 2


low_level_scf_method='RKS'
high_level_scf_method='CCSD'
E_convergence_tol = 1e-6
basis = 'STO-3G' #'augccpvdz' # '6-31g' #'STO-3G'
unit= 'angstrom'
pyscf_print_level=1
memory=8000
charge=0
spin=0

low_level_xc_functional = 'lda, vwn' # 'b3lyp'
high_level_xc_functional = 'b3lyp'

phys_notation = True
run_fci= False#True
run_cisd= True#True



In [12]:
### options

# Projector
projector_method = 'huzinaga'
# projector_method = 'mu_shfit'

# localization method
localization_method = 'PipekMezey'# 'ibo', 'Boys', 'PipekMezey' 'SPADE'
THRESHOLD = 0.995

In [13]:
drive_obj = vqe_in_dft.embeddeding_SCF_driver(geometry,
                 N_active_atoms,
                 projector_method,
                 cheap_global_SCF_method='RKS', 
                 cheap_global_DFT_xc= 'lda, vwn',
                 expensive_global_DFT_xc = 'b3lyp',
                 cheap_WF_method = 'RHF',
                 expensive_WF_method = 'CCSD',
                 E_convergence_tol = 1e-6,
                 basis = basis,
                 unit= 'angstrom',
                 pyscf_print_level=1,
                 memory=8000,
                 charge=0,
                 spin=0,
                 run_fci=run_fci,
                 run_cisd=run_cisd,
                 mu_value=1e6,
                 physists_notation= False)#phys_notation)



E(RCISD) = -39.80303613703585  E_corr = -0.0762252226044578
UCISD total energy =  -39.80303613703585


In [14]:
drive_obj.Draw_molecule()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [15]:
E_DFT_low, E_DFT_high, E_Classical, E_QC, H_fermionic = drive_obj.run_experiment(localization_method, 
                                                                      orbtial_loc_threshold=THRESHOLD,
                                                                                jupyter_notebook=True)


(active_AO^2)/(all_AO^2): [1.     0.9949 0.9993 0.8768 0.8768]
threshold for active part: 0.995 

number of active MOs: 2
number of enviro MOs: 3 

N_active_elec + N_environment_elec = N_total_elec is: True
y_active + y_enviro = y_total is: True

number of e- in gamma_embedded is correct: True


RKS correction: -0.13978662819844082


global DFT calculation == seperated calculation: False
Cheap DFT energy Calculation: -37.58780931405062


Expensive DFT energy Calculation: -37.95862054109419

E(CCSD) = -13.25393265444405  E_corr = -0.0187527313116477

WF hartree fock energy matches HF embedded calc: True!


WF correction: 15.994569441179017
Expensive WF (classic) energy Calculation: -37.88005530198332

H core is NOT standard H_core




No terms in Fermionic H : 5257

expect 4 electrons
quantum state has 5 electrons 

number of electrons correct: False
superposition states: [ 124  188  628  692  841  905  908  944  964 1144 1208 1330 1354 1418
 1586 1612 1712 1730 1732 1804 1840 1858 1864 1924 1984 2161 2225 2288
 2416 2444 2500 2612 2629 2672 2756 2828 2882 2945 2948 3079 3083 3086
 3121 3124 3145 3146 3160 3203 3205 3206 3212 3224 3248 3265 3272 3334
 3337 3376 3394 3400 3458 3520 3587 3590 3596 3649 3664 3713 3716 3728
 3842 3844 3856 3904]


In [18]:
np.binary_repr(628).count('1')

5

In [8]:
# # Draw active orbitals

# list_active_orbitals = vqe_in_dft.Draw_cube_orbital(full_system_scf,
#                              xyz_string,
#                              drive_obj.C_all_localized, 
#                              drive_obj.active_MO_inds,
#                              width=400, 
#                              height=400, 
#                              jupyter_notebook=True)

# list_enviro_orbitals = vqe_in_dft.Draw_cube_orbital(full_system_scf,
#                              xyz_string,
#                              drive_obj.C_all_localized, 
#                              drive_obj.enviro_MO_inds,
#                              width=400, 
#                              height=400, 
#                              jupyter_notebook=True)

In [9]:
if run_fci:
    print('LOW level DFT in DFT error:', np.abs(E_DFT_low-drive_obj.FCI_obj.e_tot))
    print('High level DFT in DFT error:', np.abs(E_DFT_high-drive_obj.FCI_obj.e_tot))
    print('WF in DFT error:', np.abs(E_Classical-drive_obj.FCI_obj.e_tot))
    print('VQE in DFT error:', np.abs(E_QC-drive_obj.FCI_obj.e_tot))
    
if run_cisd:
    print('LOW level DFT in DFT error:', np.abs(E_DFT_low-drive_obj.CISD_obj.e_tot))
    print('High level DFT in DFT error:', np.abs(E_DFT_high-drive_obj.CISD_obj.e_tot))
    print('WF in DFT error:', np.abs(E_Classical-drive_obj.CISD_obj.e_tot))
    print('VQE in DFT error:', np.abs(E_QC-drive_obj.CISD_obj.e_tot))
    


LOW level DFT in DFT error: 0.9277134612794811
High level DFT in DFT error: 1.1506861829289647
WF in DFT error: 1.2068938036247658
VQE in DFT error: 1.206840157494966
